In [41]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings
import glob
import os

warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.float_format ='{:,.3f}'.format

In [42]:
current_date = dt.date.today()
day = current_date.day - 1 # данные по продажам мы получаем за предыдущий день
month = current_date.month
year = current_date.year

In [43]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Выгрузки из DWH\Продажи по КБК из DWH new\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

df_dwh = pd.read_excel(latest_file)

# df_dwh = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Выгрузки из DWH\Продажи по КБК из DWH new\2024-06-01_2024-06-04.xlsx')

PermissionError: [Errno 13] Permission denied: '\\\\gold585.int\\uk\\Общее хранилище файлов\\Служба аналитики\\Выгрузки из DWH\\Продажи по КБК из DWH new\\~$2024-05-01_2024-05-09.xlsx'

In [8]:
# print(latest_file)

\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Выгрузки из DWH\Продажи по КБК из DWH new\2024-05-20_2024-05-31.xlsx


In [13]:
# мы смотрим продажи со времени последнего мониторинга, для этого определяем день недели и в зависимости от этого получаем новые данные
data = dt.datetime.isoweekday(dt.date.today())
date_now = dt.datetime.now().date().strftime('%d.%m.%Y')
if data == 1:
    last_monitoring = (dt.date.today() - dt.timedelta(days=3)).strftime('%Y-%m-%d')
else:
    last_monitoring = (dt.date.today() - dt.timedelta(days=2)).strftime('%Y-%m-%d')
    
df_dwh = df_dwh[df_dwh['Дата'] >= f'{last_monitoring}']

In [14]:
df_dwh.sample(5)

,КБК,Дата,Продавец,Номер чека,ID продажи,Тов,Артикул поставщика,Количество,Номенклатура,Номер карты,Товарная группа,Товарное направление,Тип 1,Списание СберСпасибо,Группа наценки,Вес,Проба,Размер,ШК,Цена без скидки,Базовая скидка,Скидка в руб.,Название,Цена после скидки,Phone,Себестоимость,Сумма,Услуга,Общая сумма
82300,6-01-128,2024-06-04 08:34:00,Говорунова Мария Анатольевна,5166,Ch4-000536360,ТОВ1257220,10-02-0001-10839,1,Товар,9000212982201,ИФ СЕРЬГИ,ИФ,СЕРЬГИ,0.000,ИФ.БК_СР,2.388,585.000,0.000,"2,078,609,464,420.000","45,784.000",22.000,"11,599.610","СЕРЬГИ, ФИАНИТ","16,600.000",+79614058253,"7,372.930","16,600.000","3,400.000","20,000.000"
64758,6-01-135,2024-06-03 10:55:00,Плюснина Анастасия Александровна,3593,Ch4-000534767,ТОВ1261812,МТБ1,1,Футляр,9000191580658,ФУТЛЯРЫ,ФУТЛЯРЫ,ПРОЧЕЕ,0.000,NaN,0.000,NaN,0.000,NaN,65.000,0.000,65.000,ФУТЛЯР МЕШОЧЕК БРЕНД,65.000,+79281093240,NaN,65.000,0.000,65.000
92164,4-05-069,2024-06-04 13:14:00,Данилова Светлана Ярославовна,5547,Ch2-000854741,ТОВ1261812,МТБ1,1,Футляр,9000212761639,ФУТЛЯРЫ,ФУТЛЯРЫ,ПРОЧЕЕ,0.000,NaN,0.000,NaN,0.000,NaN,65.000,0.000,65.000,ФУТЛЯР МЕШОЧЕК БРЕНД,65.000,+79222522012,NaN,65.000,0.000,65.000
60767,6-04-045,2024-06-03 09:17:00,Михайлова Светлана Юрьевна,212,Ch4-000534393,ТОВ1202674,47-02-1000-29749,1,Товар,NaN,ДК ПОДВЕСКИ,ДК,БРАСЛЕТ,0.000,ДК_ДЕШЕВАЯ,0.617,585.000,0.000,"2,078,608,176,931.000","19,967.000",70.000,"4,980.000",БРАСЛЕТ КРАСНАЯ НИТЬ,"4,980.000",NaN,"2,241.600","4,980.000","1,020.000","6,000.000"
74685,1-01-251,2024-06-03 16:25:00,Смирнова Елена Геннадьевна,215,Ch3-000799551,ТОВ0832948,КО 03-00,1,Товар,9000054775382,БК КОЛЬЦА ОБРУЧ,БК,КОЛЬЦО ОБРУЧАЛЬНОЕ,0.000,ИФ.БК_ДЕШ,1.691,585.000,17.000,"2,078,610,506,550.000","41,108.000",35.000,"11,122.000",КОЛЬЦО ОБРУЧАЛЬНОЕ,"11,122.000",+79502274682,"5,860.150","11,122.000","2,278.000","13,400.000"


In [15]:
df_dwh['Товарная группа'].str.contains('ЦБ|БК').value_counts()

Товарная группа
False    21890
True      8010
Name: count, dtype: int64

In [16]:
df_dwh = df_dwh[(df_dwh['Товарное направление'].isin(['БК', 'ЦБ'])) & (df_dwh['Проба'] == 585.0)]

In [17]:
df_dwh.head()

,КБК,Дата,Продавец,Номер чека,ID продажи,Тов,Артикул поставщика,Количество,Номенклатура,Номер карты,Товарная группа,Товарное направление,Тип 1,Списание СберСпасибо,Группа наценки,Вес,Проба,Размер,ШК,Цена без скидки,Базовая скидка,Скидка в руб.,Название,Цена после скидки,Phone,Себестоимость,Сумма,Услуга,Общая сумма
52452,3-09-051,2024-06-03 01:03:00,Горбатенко Юлия Олеговна,1099,Ch2-000848842,ТОВ1019800,"НЦ18-053-3 D0,45",1,Товар,9000086736238,ЦБ ЦЕПИ,ЦБ,ЦЕПЬ,0.000,ЦБ_АВТОМАТ >3 ГРАММ,3.919,585.000,55.000,"2,078,608,177,428.000","66,360.000",11.000,0.000,ЦЕПЬ ЯКОРЬ РОД.,"29,423.500",+79140275428,"18,076.940","29,423.500","6,026.500","35,450.000"
52457,3-09-025,2024-06-03 01:07:00,Сенотрусова Ирина Анатольевна,4299,Ch2-000848851,ТОВ0980576,"НЦ12-087ПГ D0,40",1,Товар,9000085765079,ЦБ ЦЕПИ,ЦБ,ЦЕПЬ,0.000,ЦБ_ПУСТОТЕЛЫЕ <=3 ГРАММ,1.922,585.000,50.000,"2,078,609,800,105.000","36,084.000",22.000,"12,466.870",ЦЕПЬ ЛАВ ПУСТ,"13,114.000",+79243719866,"8,449.130","13,114.000","2,686.000","15,800.000"
52461,3-09-043,2024-06-03 01:07:00,Воронова Валентина Игоревна,4473,Ch2-000848829,ТОВ0832984,КО 05-00,1,Товар,9000159920076,БК КОЛЬЦА ОБРУЧ,БК,КОЛЬЦО ОБРУЧАЛЬНОЕ,0.000,ИФ.БК_СР,3.017,585.000,19.000,"2,078,607,105,388.000","58,006.000",24.000,0.000,КОЛЬЦО ОБРУЧАЛЬНОЕ,"17,596.000",+79141487060,"10,453.960","17,596.000","3,604.000","21,200.000"
52477,3-09-046,2024-06-03 01:22:00,Сопова Алина Владимировна,3856,Ch2-000848862,ТОВ0832971,КО 04-00,1,Товар,9000212848972,БК КОЛЬЦА ОБРУЧ,БК,КОЛЬЦО ОБРУЧАЛЬНОЕ,0.000,KVI,2.622,585.000,21.000,"2,078,608,885,414.000","42,966.000",70.000,0.000,КОЛЬЦО ОБРУЧАЛЬНОЕ,"10,707.000",+79145630370,"8,495.600","10,707.000","2,193.000","12,900.000"
52480,3-09-046,2024-06-03 01:22:00,Сопова Алина Владимировна,3856,Ch2-000848862,ТОВ0832970,КО 04-00,1,Товар,9000212848972,БК КОЛЬЦА ОБРУЧ,БК,КОЛЬЦО ОБРУЧАЛЬНОЕ,0.000,KVI,2.566,585.000,20.500,"2,078,606,735,037.000","42,966.000",70.000,0.000,КОЛЬЦО ОБРУЧАЛЬНОЕ,"10,707.000",+79145630370,"7,753.670","10,707.000","2,193.000","12,900.000"


In [18]:
df_dwh['Товарное направление'].unique()

array(['ЦБ', 'БК'], dtype=object)

In [19]:
df_dwh['Название'].isna().sum()

0

In [20]:
df_dwh['Название'] = df_dwh['Название'].fillna('золотое изделие')

In [21]:
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('каучук')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('шнурок')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('текстиль')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('хлопок')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('шелк')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('на нитке')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('кожа')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('жемчуг')))]

In [22]:
df_dwh['Дата'] = pd.to_datetime(df_dwh['Дата'], format='%Y-%m-%d')
df_dwh['Дата'] = pd.to_datetime(df_dwh['Дата'].apply(lambda x: x.strftime("%Y-%m-%d")))

In [23]:
filtr_date = df_dwh['Дата'].max()

In [24]:
df_dwh['Дата'].min()

Timestamp('2024-06-03 00:00:00')

In [25]:
df_dwh = df_dwh.query("`Дата` <= @filtr_date")

In [26]:
df_dwh['Дата'].unique()

<DatetimeArray>
['2024-06-03 00:00:00', '2024-06-04 00:00:00']
Length: 2, dtype: datetime64[ns]

In [27]:
df_dwh['Товарное направление'].value_counts()

Товарное направление
БК    2666
ЦБ    2238
Name: count, dtype: int64

In [28]:
df_dwh.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4904 entries, 52452 to 97921
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   КБК                   4893 non-null   object        
 1   Дата                  4904 non-null   datetime64[ns]
 2   Продавец              4904 non-null   object        
 3   Номер чека            4904 non-null   int64         
 4   ID продажи            4904 non-null   object        
 5   Тов                   4904 non-null   object        
 6   Артикул поставщика    4904 non-null   object        
 7   Количество            4904 non-null   int64         
 8   Номенклатура          4904 non-null   object        
 9   Номер карты           4683 non-null   object        
 10  Товарная группа       4904 non-null   object        
 11  Товарное направление  4904 non-null   object        
 12  Тип 1                 4904 non-null   object        
 13  Списание СберСпаси

In [29]:
tn = df_dwh.groupby(by = 'Товарная группа', as_index= False)[['Себестоимость', 'Вес', 'Общая сумма']].agg('sum')

In [30]:
tn['Наценка продажи оффлайн'] =  tn['Общая сумма'] / tn['Себестоимость'] - 1

In [31]:
tn = tn[~tn['Товарная группа'].str.contains('ОПТ БК|БК ПЕЧАТКИ|ОПТ КОНГО|ОПТ ОБРУЧИ')]

In [32]:
tn['price_per_gramm'] = ((tn['Общая сумма'] / tn['Вес']).round(0))

In [33]:
tn['price_per_gramm'] = tn['price_per_gramm'].replace(',', '')
tn['price_per_gramm'] = tn['price_per_gramm'].astype('int')

In [34]:
tn

,Товарная группа,Себестоимость,Вес,Общая сумма,Наценка продажи оффлайн,price_per_gramm
0,БК КОЛЬЦА,"757,441.640",288.868,"1,901,056.280",1.510,6581
1,БК КОЛЬЦА ОБРУЧ,"8,038,793.150","2,715.022","17,159,113.500",1.135,6320
3,БК ПОДВЕС ДЕКОР,"550,590.220",184.116,"1,325,205.000",1.407,7198
4,БК ПОДВЕС КУЛЬТ,"2,546,927.580",824.351,"5,963,869.300",1.342,7235
5,БК СЕРЬГИ,"2,293,118.420",744.548,"5,462,206.000",1.382,7336
9,ЦБ БРАСЛЕТЫ,"8,138,955.170","2,442.915","16,993,070.480",1.088,6956
10,ЦБ ЦЕПИ,"17,427,830.270","5,432.433","38,213,552.180",1.193,7034


In [35]:
print('Средняя цена продажи оффлайн: ', (tn['Общая сумма'].sum() / tn['Вес'].sum()).round(0))

Средняя цена продажи оффлайн:  6889.0


In [36]:
tn.columns = ['tg', 'cost', 'weight', 'price', 'Наценка продажи оффлайн', 'price_per_gramm']

In [37]:
tn.to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sales_offline\offline_sales_{date_now}.xlsx', index = False)

In [38]:
tn = tn.drop(columns=['cost', 'weight', 'price', 'Наценка продажи оффлайн'])
tn.columns = ['tg', '585*Золотой оффлайн продажи']

In [40]:
pd.read_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\grouped_data\result_{date_now}.xlsx').merge(tn, how = 'left', on = 'tg').to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\grouped_data\result_{date_now}.xlsx', index=False)